# Competição kaggle Titanic
O notebook foi desenvolvido com base na playlist do canal do YouTUbe [Canal Mario Filho](https://www.youtube.com/watch?v=K86voC1gRIk&list=PLwnip85KhroW8Q1JSNbgl06iNPeC0SDkx&index=3).

# Imports

In [101]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

# Load Data

In [ ]:
train = pd.read_csv('../content/train.csv') # Train Data
test = pd.read_csv('../content/test.csv') # Test Data

# Cleaning the missing data

## Train data

In [ ]:
# Visualizing the missin data
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
sex_binary       0
dtype: int64

In [ ]:
train['Age'] = train['Age'].fillna(-1)

train['Age'].isnull().sum()

0

## Test data

In [ ]:
# Visualizing the missin data
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
sex_binary       0
dtype: int64

In [ ]:
test['Age'] = test['Age'].fillna(-1)
test['Age'].isnull().sum()

0

# Generating the Model

In [ ]:
model = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=0)

## Create Variables
- Changing passagers gender for numbers:
    - female = 1
    - male = 0

In [57]:
train['sex_binary'] = train['Sex'].map(lambda x: 1 if x == 'female' else 0)

variables_train = ['sex_binary', 'Age']

In [58]:
# Creating the variables for model
x = train[variables_train]    # Param for prediction
y = train['Survived']   # What we want to predict

(891, 2)

# Validation

## Split data for training and test
- Create the variables for training and valid the data
    - Training: x_train, y_train
    - Validation: x_valid, y_valid

In [86]:
np.random.seed(0)
x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.5)

In [87]:
# Fit model in training data
model.fit(x_train, y_train)

RandomForestClassifier(n_jobs=-1, random_state=0)

# Croos Validation

## Creatinfg KFold object

In [124]:
results = []
for rep in range(10):
    print(f"Reptition count: {rep}")
    kf = KFold(2, shuffle=True, random_state=rep)
    for lines_train, lines_valid in kf.split(x):
        print("Train", lines_train.shape[0], "\nValid", lines_valid.shape[0])

        # Split data
        x_train, x_valid = x.iloc[lines_train], x.iloc[lines_valid]
        y_train, y_valid = y.iloc[lines_train], y.iloc[lines_valid]

        # Create model and fit model
        model = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=0)
        model.fit(x_train, y_train)

        # Predict model
        model_predict = model.predict(x_valid)

        # Creating accuracy
        acc = np.mean(y_valid == model_predict)
        results.append(acc)    # Salving accuracy data

        # Visualizing the result
        print(f"Accuracy: {acc} \n")

Reptition count: 0
Train 445 
Valid 446
Accuracy: 0.7713004484304933 

Train 446 
Valid 445
Accuracy: 0.7797752808988764 

Reptition count: 1
Train 445 
Valid 446
Accuracy: 0.7443946188340808 

Train 446 
Valid 445
Accuracy: 0.7955056179775281 

Reptition count: 2
Train 445 
Valid 446
Accuracy: 0.7757847533632287 

Train 446 
Valid 445
Accuracy: 0.7887640449438202 

Reptition count: 3
Train 445 
Valid 446
Accuracy: 0.7533632286995515 

Train 446 
Valid 445
Accuracy: 0.7573033707865169 

Reptition count: 4
Train 445 
Valid 446
Accuracy: 0.7354260089686099 

Train 446 
Valid 445
Accuracy: 0.7415730337078652 

Reptition count: 5
Train 445 
Valid 446
Accuracy: 0.7219730941704036 

Train 446 
Valid 445
Accuracy: 0.7056179775280899 

Reptition count: 6
Train 445 
Valid 446
Accuracy: 0.7757847533632287 

Train 446 
Valid 445
Accuracy: 0.7303370786516854 

Reptition count: 7
Train 445 
Valid 446
Accuracy: 0.7040358744394619 

Train 446 
Valid 445
Accuracy: 0.7348314606741573 

Reptition count:

In [126]:
np.mean(results)

0.7550007557817302

## Predict model

In [91]:
# Predict with function (train_test_split)
p = model.predict(x_valid)

In [96]:
# Predict With param 'All women survived'
women_survived = (x_valid['sex_binary'] == 1).astype(np.int64)

# Applying the model in the test

##  Create Variables
- The same Changing in train table

In [88]:
test['sex_binary'] = test['Sex'].map(lambda x: 1 if x == 'female' else 0)

variables_test = ['sex_binary', 'Age']

## Test Model

In [89]:
x_prev = test[variables_test]
x_prev.head()

,sex_binary,Age
0,0,34.5
1,1,47.0
2,0,62.0
3,0,27.0
4,1,22.0


# Saving the result for Kaggle competition

In [ ]:
submition = pd.Series(p, index=test['PassengerId'], name='Survived')

In [ ]:
submition.to_csv('first_model', header=True)

In [ ]:
!head -n10 ../content/first_model

PassengerId,Survived
892,0
893,1
894,0
895,1
896,1
897,0
898,1
899,0
900,1
